In [ ]:
import sys

sys.path.insert(0, '/iopsstor/scratch/cscs/stefschu/DSM500_FPR/modulus-baseline')

In [2]:
import torch
from omegaconf import OmegaConf

from modulus.models.graph_cast_net import GraphCastNet


device = torch.device('cuda', 1)

cp_dict = torch.load(
    '/iopsstor/scratch/cscs/stefschu/DSM500_FPR/modulus-baseline/runs/RUN01/SEED21/checkpoint/model.iter003750.pth', 
    map_location=torch.device('cuda', index=1),
    weights_only=True
)

cfg = OmegaConf.create(cp_dict["cfg"])

model = GraphCastNet(cfg=cfg, device=device)

weights = cp_dict["model"]
model.load_state_dict(weights)
model.eval()

GraphCastNet(
  (encoder_embedder): GraphCastEncoderEmbedder(
    (grid_input_mlp): MeshGraphMLP(
      (model): Sequential(
        (0): Linear(in_features=31, out_features=512, bias=True)
        (1): SiLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): LayerNorm()
      )
    )
    (m2m_node_mlp): MeshGraphMLP(
      (model): Sequential(
        (0): Linear(in_features=3, out_features=512, bias=True)
        (1): SiLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): LayerNorm()
      )
    )
    (g2m_edge_mlp): MeshGraphMLP(
      (model): Sequential(
        (0): Linear(in_features=4, out_features=512, bias=True)
        (1): SiLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): LayerNorm()
      )
    )
    (m2m_edge_mlp): MeshGraphMLP(
      (model): Sequential(
        (0): Linear(in_features=4, out_features=512, bias=True)
        (1): SiLU()
        (2): Linear(in_features=512, out

In [3]:
from modulus.datapipes.era5_hdf5 import ERA5HDF5Datapipe

datapipe = ERA5HDF5Datapipe(
    cfg=cfg,
    dataset_folder="out_of_sample",
    num_output_steps=24,
    model=model,
    iterator={
        "initial_epoch_idx": 0,
        "initial_sample_idx": 55
    },
    prefetch_queue_depth=1,
    num_threads=1,
    
    shuffle=False,
    seed=0, # No use given that shuffle=False
    device=device,
    rank=0,
    world_size=1,
)

In [4]:
sample = next(iter(datapipe))
sample["data"].shape

torch.Size([25, 31, 721, 1440])

In [ ]:
sample["idx_in_epoch"], sample["global_sample_id"] 

(55, 55)

Process SpawnProcess-1:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 462, in worker
    scheduled, shm_chunk = worker_context.get_task()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 434, in get_task
    scheduled_meta = self.task_receiver.get_task()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 132, in get_task
    recv = self.queue.get()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/shared_queue.py", line 171, in get
    waited = self._wait_for_samples()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/shared_queue.py", line 99, in _wait_for_samples
    self.cv_not_e

In [5]:
sample.keys()


dict_keys(['epoch_idx', 'idx_in_epoch', 'global_sample_id', 'data', 'timestamps'])

In [16]:
from datetime import datetime 

for i, batch in enumerate(datapipe):
    print(i, batch["data"].shape, batch["epoch_idx"], batch["global_sample_id"], datetime.fromtimestamp(batch["timestamps"][0]))
    
    if i == 10:
        break

batch.keys()

0 torch.Size([25, 31, 721, 1440]) 0 55 2018-01-14 18:00:00
1 torch.Size([25, 31, 721, 1440]) 0 56 2018-01-15 00:00:00
2 torch.Size([25, 31, 721, 1440]) 0 57 2018-01-15 06:00:00
3 torch.Size([25, 31, 721, 1440]) 0 58 2018-01-15 12:00:00
4 torch.Size([25, 31, 721, 1440]) 0 59 2018-01-15 18:00:00
5 torch.Size([25, 31, 721, 1440]) 0 60 2018-01-16 00:00:00
6 torch.Size([25, 31, 721, 1440]) 0 61 2018-01-16 06:00:00
7 torch.Size([25, 31, 721, 1440]) 0 62 2018-01-16 12:00:00
8 torch.Size([25, 31, 721, 1440]) 0 63 2018-01-16 18:00:00
9 torch.Size([25, 31, 721, 1440]) 0 64 2018-01-17 00:00:00
10 torch.Size([25, 31, 721, 1440]) 0 65 2018-01-17 06:00:00


dict_keys(['epoch_idx', 'idx_in_epoch', 'global_sample_id', 'data', 'timestamps'])

In [17]:
batch["data"].dtype, batch["data"].device, batch["data"].nbytes / 1024**2

(torch.bfloat16, device(type='cuda', index=1), 1534.7213745117188)

In [19]:
with torch.no_grad():
    output = model(batch["data"][0])

output.shape

torch.Size([21, 721, 1440])

Process SpawnProcess-5:
Process SpawnProcess-4:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 462, in worker
    scheduled, shm_chunk = worker_context.get_task()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 434, in get_task
    scheduled_meta = self.task_receiver.get_task()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 132, in get_task
    recv = self.queue.get()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/shared_queue.py", line 171, in get
    waited = self._wait_for_samples()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/shared_queue.py", line 99, in _wait_for_s